In [2]:
import pykafka
import json
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime,timezone
from tweepy.streaming import StreamListener
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#TWITTER API CONFIGURATIONS
consumer_key = ### Give the consumer key ###
consumer_secret = ### Give the consumer secret key ###
access_token = ### Give the access token ###
access_secret = ### Give the access secret token ###

#TWITTER API AUTH
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

SA = SentimentIntensityAnalyzer()

def clean_tweet_extract(data):
    
    rawtweet = json.loads(data)
    tweet={}
    tweet["date"] = datetime.strptime(rawtweet["created_at"],'%a %b %d %H:%M:%S %z %Y')\
        .replace(tzinfo=timezone.utc).astimezone(tz=None).strftime('%Y-%m-%d %H:%M:%S')
    tweet["user"] = rawtweet["user"]["screen_name"]
    if "extended_tweet" in rawtweet:
        tweet["text"] = rawtweet["extended_tweet"]["full_text"]
    else:
        tweet["text"] = rawtweet["text"]
        
    #tweet["country"] = rawtweet['place']['country']
    #tweet['country_code'] = rawtweet['place']['country_code']
    
    tweet['reply_count'] = rawtweet['reply_count']
    tweet['retweet_count'] = rawtweet['retweet_count']
    tweet['favorite_count'] = rawtweet['favorite_count']
    #Sentiment Analysis
    sentiment_dict = SA.polarity_scores(tweet['text'])
    tweet['sentiment_score'] = sentiment_dict['compound']
    
    return json.dumps(tweet)

#Twitter Stream Listener
class KafkaPushListener(StreamListener):          
    def __init__(self):
        #localhost:9092 = Default Zookeeper Producer Host and Port Adresses
        self.client = pykafka.KafkaClient("localhost:9092")

        #Get Producer that has topic name is Twitter
        self.producer = self.client.topics[bytes("kafkaspark", "ascii")].get_producer()
  
    def on_data(self, data):
        #Producer produces data for consumer
        #Data comes from Twitter
        processed_data = clean_tweet_extract(data)
        self.producer.produce(bytes(processed_data, "ascii"))
        print(processed_data)
        return True
                                                                                                                                           
    def on_error(self, status):
        print(status)
        return True

#Twitter Stream Config
twitter_stream = Stream(auth, KafkaPushListener())

#Produce Data that has Game of Thrones hashtag (Tweets)
twitter_stream.filter(track=['#Trump'])

{"date": "2020-12-14 12:25:55", "user": "trennahibner", "text": "RT @drewmorgcomedy: DIAMOND DREW MORGAN IS TRUMP'S LAST CHANCE\n\n- \ud83d\udc8eDrew Morgan is the ELECTION LAW expert who can #MakeAmericaGreatAgain\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.7065}
{"date": "2020-12-14 12:25:57", "user": "snug_knot546", "text": "RT @GoritanJapan: \u30d1\u30a6\u30a8\u30eb\u5f01\u8b77\u58eb\u6012\u3063\u3066\u307e\u3059\u3002\n\n\u30d0\u30a4\u30c7\u30f3\u6c0f\u306f\u5927\u7d71\u9818\u306b\u7d76\u5bfe\u306a\u308c\u307e\u305b\u3093\u3067\u3057\u305f\u3002\n\u9078\u6319\u306f\u5927\u898f\u6a21\u306a\u8a50\u6b3a\u3067\u3057\u305f\n\u30c8\u30e9\u30f3\u30d7\u306f\u52dd\u3061\u307e\u3057\u305f\n\u56fd\u6c11\u306f\u305d\u308c\u3092\u77e5\u3063\u3066\u3044\u307e\u3059\ud83c\uddfa\ud83c\uddf8\n\u30c8\u30e9\u30f3\u30d7\n\u30b9\u30ab\u30d3\u30ce\n\u30a6\u30c3\u30c9\n\u30d5\u30ea\u30f3\n\u79c1\u305f\u3061\u306f\u4e0d\u6b63\u9078\u6319\u3092\u3082\u3046\u8a8d

{"date": "2020-12-14 12:26:50", "user": "mapyntonga", "text": "RT @CorentinSellin: Ce lundi 14/12,#JoeBiden sera \u00e9lu pr\u00e9sident des Etats-Unis par le coll\u00e8ge des grands \u00e9lecteurs d\u00e9sign\u00e9s par le vote pop\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-12-14 12:26:52", "user": "GayleneSass", "text": "RT @ABcustomcoach: #POTUS is always 3steps ahead. It\u2019s gonna be a crazy week #Patriots stay safe and enjoy the show. #Trump #Military", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.5719}
{"date": "2020-12-14 12:26:53", "user": "espritdefamille", "text": "RT @Trump_Fact_News: \ud83d\udea8 Le d\u00e9cret sign\u00e9 par le pr\u00e9sident #Trump concernant l'intervention \u00e9trang\u00e8re dans les \u00e9lections am\u00e9ricaines pourrait\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-12-14 12:26:55", "user": "

{"date": "2020-12-14 12:28:26", "user": "AnitaBeavers", "text": "#Trump   has Texas as his stronghold. #Biden is weakened by DC", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": -0.2023}
{"date": "2020-12-14 12:28:39", "user": "thrWSMW", "text": "You've got to be kidding, EVERY DAY DEATHS are the equivalent of 9/11 or higher, half a million DEAD projected by February.\n#Trump KNEW yet said it was a #Hoax\nAn American dying every 30 seconds unprecedented results, alright. Due to the unprecedented lack of leadership.", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": -0.7925}
{"date": "2020-12-14 12:28:48", "user": "Gilas79201942", "text": "RT @kleensamsonite: #Trump \n\nNous ne serons pas\n intimid\u00e9.\n Les patriotes am\u00e9ricains sont\n marre de la corruption.\n Nous allons reprendre e\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-12-14 12:28:51", "user": "JeedBe

{"date": "2020-12-14 12:29:58", "user": "dimarmart", "text": "RT @GloriaElizo: \u00bfDeber\u00eda sorprendernos -adem\u00e1s de indignarnos- la decisi\u00f3n de #Trump de morir matando?\n\n\u00bfSer\u00e1 que la Historia nos ense\u00f1\u00f3 q\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-12-14 12:30:00", "user": "fdparedese", "text": "RT @ReutersLatam: Opositores al presidente Donald #Trump y manifestantes del grupo de derechista #ProudBoys se enfrentan en Washington (ing\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-12-14 12:30:00", "user": "TheIranPulse", "text": "The #Iranian president wants #Biden to reverse #Trump policies, not start a new round of talks; the #US president-elect makes clear US regional partners will also have their say  https://t.co/2SgIwUHulN", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.3818}
{"date": "2020-

{"date": "2020-12-14 12:30:32", "user": "FrancescoAwaken", "text": "RT @fdragoni: Secondo #Gallup l'83% degli elettori #GOP ed il 30% degli elettori #DEM pensano che le elezioni siano state truccate. #Trump\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-12-14 12:30:33", "user": "Icingdeathe", "text": "Oh, I've learned plenty over these last hellish four yrs. \n#Trump is already threatening in tweets that those who don't support his continuing coup will be sorry come Jan 5. \n\nSomeone needs to hide the Kool-Aid and fast. \n\n#TrumpDerangementSyndrome #TrumpMeltdown @realDonaldTrump", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": -0.897}
{"date": "2020-12-14 12:30:37", "user": "Obertson", "text": "Judge allows fraud suit against Trump, family and company to proceed https://t.co/3xDjFnDbUq via @Yahoo @GOP #GOP #ElectionResults2020 #Trump #POTUS #America #UnitedStates #Kraken #Trending #trumplies #D

...

{"date": "2020-12-14 13:43:51", "user": "kk131066", "text": "RT @kk131066: 3\n\nPeople cannot accept #lies &amp;\nhope for a bright future\n\n#people either stand up for #Truth\nor\nhide behind lies\n\nPeople cann\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.5787}
{"date": "2020-12-14 13:43:56", "user": "FRED3518", "text": "RT @gDesFaits: \"Le niveau d'ing\u00e9rence \u00e9trang\u00e8re ... est plus que suffisant pour d\u00e9clencher le d\u00e9cret pr\u00e9sidentiel de 2018 qui lui donne tou\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-12-14 13:44:02", "user": "FRED3518", "text": "RT @gDesFaits: \ud83c\uddfa\ud83c\uddf8Executive Order | The White House \ud83c\uddfa\ud83c\uddf8\n\nD\u00e9cret ex\u00e9cutif sur l'imposition de certaines sanctions en cas d'ing\u00e9rence \u00e9trang\u00e8re\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date

{"date": "2020-12-14 13:45:47", "user": "ladylight33", "text": "What's up with that!! We can bring in suit cases of massive fraud evidence and massive affidavits!! Our suit cases don't count?? WTF??? https://t.co/YJN4RmsFVe", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": -0.5128}
{"date": "2020-12-14 13:45:50", "user": "CalvinSwine905", "text": "@DWUhlfelderLaw @POTUS @FLOTUS @CNN \nYes, I remember the #Trump #Hoax trending.\ud83d\ude37\n#WH https://t.co/5h8Jdj7HDv", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.1531}
{"date": "2020-12-14 13:45:53", "user": "PopsieWagga1", "text": "RT @TobiasTheGeek: Thousands gather in Washington D.C. to show support for #Trump\n\nIt should read:\n\u201cThousands gather in Washington D.C. to\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.4019}
{"date": "2020-12-14 13:45:57", "user": "macaronjoe", "text": "The #Trump admin's #US sanctions made Bas

{"date": "2020-12-14 13:47:02", "user": "AssezJohnny", "text": "#vaccine is a code word.\n#ET in contact with government.\n\ud83d\ude0d\ud83d\ude0e\ud83d\ude0d\ud83d\ude0d\ud83d\ude0d\n#Trump won", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.9565}
{"date": "2020-12-14 13:47:16", "user": "mastok40", "text": "RT @Trump_Fact_News: Quoi qu'il arrive le pr\u00e9sident #Trump a confirm\u00e9 que les cinq violeurs et assassins d'enfants en attente de peine de m\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-12-14 13:47:21", "user": "marauder_red", "text": "#SidneyPowell on American Thought Leaders or Epoch or it\u2019s 3am and we better pull this through \u2018cause #Trump won and we know it! \n\nhttps://t.co/erEMFLCBCV", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.784}
{"date": "2020-12-14 13:47:21", "user": "javabird", "text": "RT @contactlight69: Trump mislead 

{"date": "2020-12-14 13:48:42", "user": "CompetingMonies", "text": "@IrmaBel53130008 @DigiDiva2020 @realDonaldTrump #StopTheSteal #FightBack #FightForTrump #FightBackForAmerica #MAGA2020 #MAGA #TRUMP #HoldTheLinePatriots #HoldTheLine #FakeNews #FightBack #IAMAPATRIOT #MarchForTrump #TREASON #Overturn #krackenwood #NotOver #patriots #millionmagamarch https://t.co/eRSyU9k1R1", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": -0.5622}


KeyboardInterrupt: 

In [1]:
from platform import python_version

print(python_version())

3.8.8
